# <center> **Base de données (Films)**

## **Création de la base de données**

Des fichiers csv contenant les données ont été générés lors de la phase précédente de scrapping, nous allons maintenant lire ces fichiers csv dans des dataFrame Pandas et nettoyer les données puis les insérer dans une base de données SQL.


## **Création du schéma relationnel**

Online tool: https://drawdb.vercel.app/editor<br>

Une difficulté rencontrée est de faire apparaître les films similaires à un film dans la base de données, en effet cette situation est une relation "Many to many" où la table se référence elle-même, c'est un cas fréquent dans les réseaux sociaux où un utilisateur va avoir des amis eux-mêmes utilisateurs.<br>
Deux situations sont observées : <br>
- les relations symétriques (l'utilisateur Alice est amis avec l'utilisateur Bob implique nécessairement que Bob est ami d'Alice),
- les relations asymétriques (Alice est amis avec Bob mais Bob n'est pas forcément ami avec Alice).

Dans notre situation la relation est .....????

Pour représenter cette relation ....





**About Self many-to-many relationship**: https://stackoverflow.com/questions/17128472/many-to-many-on-same-table

Relations n-n

Remarque :
La table "reviews" sert de table de jonction entre les films et les utilisateurs, en effet la relation movies-users est une relation n-n car un utilateur peut écrire des avis pour plusieurs films et un film possède des avis de plusieurs utilisateurs.


